In [ ]:
import pandas as pd
import os

import config

from utils.evaluation_utils import print_ensemble_results

In [ ]:
PATH_TO_ANOMSCORES = config.PATH_TO_ANOM

In [ ]:
path_to_scores = os.path.join(PATH_TO_ANOMSCORES, 'ss')

In [ ]:
print_ensemble_results(path_to_anom_scores=path_to_scores, epoch=400, stage='ss', metric='centre_mean', meta_data_dir=None, get_oarsi_results=False, model_name_prefix="mod_1")